In [2]:
import numpy as np
from oceanmixedlayers import oceanmixedlayers
import matplotlib.pyplot as plt

import gsw as gsw
import xarray as xr
from glob import glob
import os
import warnings
from oct2py import octave
%matplotlib inline


def IsVarThere(hndl,VAR):
    CHECK = [A for A in hndl.data_vars  if A==VAR]
    return len(CHECK)>0

In [ ]:
#Some deep ones
#PATH = '/net3/bgr/Datasets/Argo/202011-ArgoData/dac/aoml/3900331/'
#PATH = '/net3/bgr/Datasets/Argo/202011-ArgoData/dac/aoml/3900451/'
PATH = '/net3/bgr/Datasets/Argo/202011-ArgoData/dac/aoml/3900660/'
#Holte and Talley sample
#PATH = '/net3/bgr/Datasets/Argo/202011-ArgoData/dac/aoml/3900621/'


PROFFILE = glob(PATH+'/*_prof.nc')

with xr.open_dataset(PROFFILE[0]) as hndl:

    if ( (IsVarThere(hndl,'TEMP'))&(IsVarThere(hndl,'PSAL'))):
        print('OK')
        NP = hndl.N_PROF.size
        NZ = hndl.N_LEVELS.size
        Z_c_2d       = np.zeros([NP,NZ])+np.NaN
        dZ_2d        = np.zeros([NP,NZ])+np.NaN
        P_c_2d       = np.zeros([NP,NZ])+np.NaN
        CT_c_2d      = np.zeros([NP,NZ])+np.NaN
        PSal_c_2d    = np.zeros([NP,NZ])+np.NaN
        dRho0dz_c_2d = np.zeros([NP,NZ])+np.NaN
        Rho0_c_2d    = np.zeros([NP,NZ])+np.NaN
        GOOD         = np.zeros([NP],dtype=bool)
        for p in range(NP):
            DayQC = float(hndl.JULD_QC[p])
            PosQC = float(hndl.POSITION_QC[p])
            if (np.max(DayQC==np.array([1,2,5,8]))
                and 
                np.max(PosQC==np.array([1,2,5,8]))):
                
                #QC
                SALTQC = np.array(hndl.PSAL_QC[p,:],dtype=float)
                TEMPQC = np.array(hndl.TEMP_QC[p,:],dtype=float)
                PRESQC = np.array(hndl.PRES_QC[p,:],dtype=float)
                
                #Computes in-situ density from T&S, T, or S
                LI = (((SALTQC==1)|(SALTQC==2)|(SALTQC==5)|(SALTQC==8))
                      &
                      ((TEMPQC==1)|(TEMPQC==2)|(TEMPQC==5)|(TEMPQC==8))
                      &
                      ((PRESQC==1)|(PRESQC==2)|(PRESQC==5)|(PRESQC==8)))
                if (np.sum(LI)>20):
                    Pressure = np.array(hndl.PRES[p,:][LI],dtype=float)
                    PSal = np.array(hndl.PSAL[p,:].values[LI],dtype=float)
                    Temp = np.array(hndl.TEMP[p,:].values[LI],dtype=float)
                    GOOD[p]=True
                else:
                    GOOD[p]=False

                if GOOD[p]:
                    Rho = gsw.density.rho_t_exact(PSal,Temp,Pressure)
                    CT = gsw.conversions.CT_from_rho(Rho,PSal,Pressure)[0]
                    Rho0 = gsw.density.rho(PSal,CT,0)
                    Z = gsw.conversions.z_from_p(Pressure, 
                                                 np.array(hndl.LATITUDE[p],dtype=float), 
                                                 geo_strf_dyn_height=0, 
                                                 sea_surface_geopotential=0)
                    P_i = np.array([0.]+list(Pressure))
                    Z_i = np.array([0.]+list(Z))
                    dZ = (Z_i[:-1]-Z_i[1:])
                    
                    if (np.min(dZ)<=0 or np.max(dZ)>100):
                        GOOD[p]=False
                    else:
                        gsdh = gsw.geostrophy.geo_strf_dyn_height(PSal, 
                                                                  CT, 
                                                                  Pressure, 
                                                                  p_ref=0, 
                                                                  axis=0, 
                                                                  max_dp=1.0, 
                                                                  interp_method='pchip')
                        Z = gsw.conversions.z_from_p(Pressure, 
                                                     np.array(hndl.LATITUDE[p],dtype=float), 
                                                     geo_strf_dyn_height=gsdh, 
                                                     sea_surface_geopotential=0)
                        Z_i = np.array([0.]+list(Z))
                        dZ = (Z_i[:-1]-Z_i[1:])
                        Z = gsw.conversions.z_from_p(Pressure, 
                                                     np.array(hndl.LATITUDE[p],dtype=float), 
                                                     geo_strf_dyn_height=gsdh, 
                                                     sea_surface_geopotential=0)
                        CT_i = np.array([CT[0]]+list(CT))
                        PSal_i = np.array([PSal[0]]+list(PSal))
                        Rho0_i = np.array([Rho0[0]]+list(Rho0))

                        P_c = 0.5*(P_i[1:]+P_i[:-1])
                        Z_c = 0.5*(Z_i[1:]+Z_i[:-1])
                        CT_c = 0.5*(CT_i[1:]+CT_i[:-1])
                        PSal_c = 0.5*(PSal_i[1:]+PSal_i[:-1])
                        Rho0_c = 0.5*(Rho0_i[1:]+Rho0_i[:-1])
                        dRho0dz_c = (Rho0_i[...,:-1]-Rho0_i[...,1:])/dZ

                        LG = len(Z_c)
                        Energy = 25/9.81
                        CT_c_2d[p,:LG] = CT_c
                        PSal_c_2d[p,:LG] = PSal_c
                        P_c_2d[p,:LG] = P_c
                        Z_c_2d[p,:LG] = Z_c
                        dRho0dz_c_2d[p,:LG] = dRho0dz_c
                        Rho0_c_2d[p,:LG] = Rho0_c
                        dZ_2d[p,:LG] = dZ
                                 